<font color='blue'><h1>Introdução Prática ao PyTorch</h1></font>

<font color='0471A6'><h2>Loop de Otimização no PyTorch - Exemplo com Classificação Binária</h2></font>

- O modelo é treinado / otimizado em um loop de otimização
- Cada iteração será uma época
- Cada época contém duas partes principais:
  - Loop de treinamento
    - iteração sobre o conjunto de treinamento
  - Loop de validação
    - iteração sobre o conjunto de validação

Hiperparâmetros (parâmetros ajustáveis pelo projetista). Exemplos:
- Número de Épocas
- Tamanho do Lote (Batch)
- Taxa de Aprendizagem

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test\
= train_test_split(X, y, test_size=0.25)

In [4]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

In [5]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, 
                          batch_size=10, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, 
                         batch_size=1, shuffle=True)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [7]:
net = MLP()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)                   

In [8]:
net.train()

MLP(
  (fc1): Linear(in_features=30, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [9]:
# Treinando a rede neural
for epoch in range(1000):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad() 
        outputs = net(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if epoch % 100 == 0:
      print('Epoch %d loss: %.3f' % (epoch, 
          running_loss / len(train_loader)))

Epoch 0 loss: 0.226
Epoch 100 loss: 0.233
Epoch 200 loss: 0.221
Epoch 300 loss: 0.233
Epoch 400 loss: 0.233
Epoch 500 loss: 0.232
Epoch 600 loss: 0.232
Epoch 700 loss: 0.232
Epoch 800 loss: 0.233
Epoch 900 loss: 0.233


In [10]:
net.eval() 

correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = net(inputs)
        predictions = torch.round(outputs)
        total += targets.size(0)
        correct += (predictions == 
                    targets.unsqueeze(1)).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy:.4f}")

Accuracy on test set: 0.6084


# Referências 

* Documentação do Pytorch. Disponível em https://pytorch.org 
* Tutorial oficial do Pytorch. Disponível em https://pytorch.org/tutorials/beginner/basics/intro.html 
* Introdução ao Pytorch. Canal “Programação Dinâmica”. Disponível em https://www.youtube.com/watch?v=cGxv8tOaA7I 
* PyTorch Crash Course - Getting Started with Deep Learning. Canal AssemblyAI". Disponível em https://youtu.be/OIenNRt2bjg 
